In [1]:
import tensorflow as tf
import magenta as mg
import numpy as np
import sys

from magenta.lib import tf_lib

In [2]:
LINUX = False

In [3]:
if LINUX:
    RL_PATH = "/home/natasha/Developer/magenta_my_fork/magenta/magenta/models/rl_rnn"
    DROPBOX_PATH = "/home/natasha/Dropbox (MIT)/"
else:
    RL_PATH = "/Users/natasha/Developer/magenta/magenta/magenta/models/rl_rnn"
    DROPBOX_PATH = "/Users/natasha/Dropbox (MIT)/"

In [4]:
sys.path.append(RL_PATH)
import melody_q
import rl_rnn_ops

In [10]:
# File locations
CHECKPOINT_DIR = RL_PATH + '/testdata'
CHECKPOINT_FILE = CHECKPOINT_DIR + '/model.ckpt-1994'
MIDI_PRIMER = RL_PATH + '/testdata/primer.mid'
OUTPUT_DIR = DROPBOX_PATH + 'Google/Project/rl_rnn_output'
LOG_DIR = DROPBOX_PATH + 'Google/Project/rl_rnn_output/rl_rnn_model.ckpt'
TRAINING_DATA_PATH = ''

In [6]:
# Model parameter settings
hparams = rl_rnn_ops.small_model_hparams()
PRIME_WITH_MIDI = False

In [7]:
dqn_hparams = tf_lib.HParams(random_action_probability=0.1,
                         store_every_nth=1,
                         train_every_nth=5,
                         minibatch_size=32,
                         discount_rate=0.5,
                         max_experience=100000,
                         target_network_update_rate=0.01)

## Train network

In [8]:
reload(rl_rnn_ops)
reload(melody_q)

<module 'melody_q' from 'melody_q.pyc'>

In [12]:
rl_net = melody_q.MelodyQNetwork(OUTPUT_DIR, LOG_DIR, CHECKPOINT_DIR,
                                 MIDI_PRIMER, dqn_hparams=dqn_hparams, 
                                 reward_scaler=2.0,
                                 output_every_nth=1000, 
                                 training_file_list=[TRAINING_DATA_PATH],
                                 backup_checkpoint_file=CHECKPOINT_FILE,
                                 custom_hparams=hparams,
                                 num_notes_in_melody=64,
                                 stochastic_observations=False,
                                 initialize_immediately=False)

In [13]:
rl_net.initialize_internal_models_graph_session(restore_from_checkpoint=False)

In [15]:
for v in rl_net.q_network.variables():
    print v.name

q_network/RNN/MultiRNNCell/Cell0/BasicLSTMCell/Linear/Matrix:0
q_network/RNN/MultiRNNCell/Cell0/BasicLSTMCell/Linear/Bias:0
q_network/fully_connected/weights:0
q_network/fully_connected/bias:0


In [16]:
# read contents of checkpoint file
reader = tf.train.NewCheckpointReader(CHECKPOINT_FILE)
[str(x) for x in reader.debug_string().decode("utf-8").split('\n')]

['rnn_model/rnn_model/fully_connected/weights/Adagrad (DT_FLOAT) [100,38]',
 'rnn_model/rnn_model/RNN/MultiRNNCell/Cell0/LSTMCell/W_0/Adagrad (DT_FLOAT) [138,400]',
 'rnn_model/rnn_model/RNN/MultiRNNCell/Cell0/LSTMCell/B/Adagrad (DT_FLOAT) [400]',
 'rnn_model/fully_connected/bias (DT_FLOAT) [38]',
 'rnn_model/rnn_model/fully_connected/bias/Adagrad (DT_FLOAT) [38]',
 'rnn_model/RNN/MultiRNNCell/Cell0/LSTMCell/W_0 (DT_FLOAT) [138,400]',
 'rnn_model/global_step (DT_INT32) []',
 'rnn_model/fully_connected/weights (DT_FLOAT) [100,38]',
 'rnn_model/RNN/MultiRNNCell/Cell0/LSTMCell/B (DT_FLOAT) [400]',
 '']